# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 6243, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 6243 (delta 53), reused 25 (delta 10), pack-reused 6089 (from 4)
Receiving objects: 100% (6243/6243), 400.66 MiB | 19.51 MiB/s, done.
Resolving deltas: 100% (3433/3433), done.
Updating files: 100% (490/490), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 41.5 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprec

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithKFCV/MatrixFactorization/PureSVDRecommender'

np.random.seed(42)

## Import the repository **RECsys_Challenge2024**

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'ScaledPureSVD',
    'n_folds': 5,
    'metric': 'MAP',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_ScaledPureSVD_MAP.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/working/RECsys_Challenge2024/Dataset/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/working/RECsys_Challenge2024/Dataset/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.split_functions.split_train_k_folds import split_train_k_folds

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

folds = split_train_k_folds(URM_all, k=config['n_folds'])

In [12]:
from Recommenders.MatrixFactorization.PureSVDRecommender import ScaledPureSVDRecommender

def objective_function_ScaledPureSVD(optuna_trial):
    
    full_hyperp = {
                    'num_factors': optuna_trial.suggest_int('num_factors', 1, 2000, log=True),
                    'scaling_items': optuna_trial.suggest_float('scaling_items', 1e-2, 2, log=True),
                    'scaling_users': optuna_trial.suggest_float('scaling_users', 1e-2, 2, log=True),
                  }        
    
    validation_results = []
    
    for fold_idx, (URM_train_fold, URM_validation_fold) in enumerate(folds):
    
        recommender_instance = ScaledPureSVDRecommender(URM_train_fold)
        recommender_instance.fit(**full_hyperp)
        
        evaluator = EvaluatorHoldout(URM_validation_fold, cutoff_list=[10])
        result_df, _ = evaluator.evaluateRecommender(recommender_instance)
        
        validation_results.append(result_df.loc[10]["MAP"])
    
    return np.mean(validation_results)

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_ScaledPureSVD, n_trials=30)

[I 2025-01-05 14:40:53,105] Using an existing study with name 'hyperparameters_tuning_ScaledPureSVD_MAP' instead of creating a new one.


ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 1.57 min
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 2.14 min. Users per second: 277
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 1.59 min
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 2.10 min. Users per second: 282
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 1.57 min
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 2.12 min. Users per second: 279
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done

[I 2025-01-05 14:59:18,424] Trial 20 finished with value: 0.029143548373414728 and parameters: {'num_factors': 1247, 'scaling_items': 0.365907052857316, 'scaling_users': 0.0567771082719638}. Best is trial 14 with value: 0.03217330506858227.


ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 1.58 min
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 2.14 min. Users per second: 277
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 1.61 min
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 2.17 min. Users per second: 273
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 1.59 min
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 2.19 min. Users per second: 270
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done

[I 2025-01-05 15:18:00,357] Trial 21 finished with value: 0.031177530979560976 and parameters: {'num_factors': 1255, 'scaling_items': 0.2795419608325479, 'scaling_users': 0.04536382719589303}. Best is trial 14 with value: 0.03217330506858227.


...

[I 2025-01-05 20:18:24,087] Trial 47 finished with value: 0.02899451100673976 and parameters: {'num_factors': 334, 'scaling_items': 0.010215067653437875, 'scaling_users': 0.0754131233768062}. Best is trial 46 with value: 0.03699934341616755.


ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 14.29 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 52.84 sec. Users per second: 673
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 16.98 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 52.97 sec. Users per second: 672
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 15.21 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 53.02 sec. Users per second: 671
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition..

[I 2025-01-05 20:24:03,790] Trial 48 finished with value: 0.022422861179835548 and parameters: {'num_factors': 195, 'scaling_items': 0.020657461817122903, 'scaling_users': 0.19696534147656983}. Best is trial 46 with value: 0.03699934341616755.


ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 1.57 min
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 2.23 min. Users per second: 265
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 1.55 min
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 2.19 min. Users per second: 270
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 1.56 min
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 2.19 min. Users per second: 270
ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done

[I 2025-01-05 20:42:47,701] Trial 49 finished with value: 0.029101180043909724 and parameters: {'num_factors': 1089, 'scaling_items': 0.012508623971645, 'scaling_users': 0.694091456002227}. Best is trial 46 with value: 0.03699934341616755.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = ScaledPureSVDRecommender(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

ScaledPureSVDRecommender: Computing SVD decomposition...
ScaledPureSVDRecommender: Computing SVD decomposition... done in 1.63 min


# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/working/RECsys_Challenge2024/Dataset/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_ScaledPureSVD_MAP.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithKFCV/MatrixFactorization/PureSVDRecommender/ScaledPureSVDRecommender/OptimizingMAP/best_params_ScaledPureSVD_MAP.json' updated successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithKFCV/MatrixFactorization/PureSVDRecommender/ScaledPureSVDRecommender/OptimizingMAP/history_ScaledPureSVD_MAP.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithKFCV/MatrixFactorization/PureSVDRecommender/ScaledPureSVDRecommender/OptimizingMAP/Submission/submission_ScaledPureSVD_MAP.csv' updated successfully.
